In [0]:
# Use the Databricks mount point for silver data
base_path = "/mnt/silver/"

# Define table configurations
tables = {
    "promotion": "promotion/",
    "item": "items/",
    "sales": "sales/",
    "supermarkets": "supermarkets/"  
}

# Create dataframes and views in a loop
dataframes = {}
for table_name, folder in tables.items():
    df = (
        spark.read
        .option("header", True)
        .option("inferSchema", True)
        .option("multiLine", True)
        .parquet(f"{base_path}{folder}")
    )
    
    # Store in dictionary for later use
    dataframes[table_name] = df
    
    # Create temp view
    df.createOrReplaceTempView(table_name)
    
    print(f"Created view: {table_name}")

promotion_df = dataframes["promotion"]
item_df = dataframes["item"]
sales_df = dataframes["sales"]
supermarkets_df = dataframes["supermarkets"]

In [0]:
# --- 1. Join Sales and Item Data (creates 'combined_1' view) ---
combined_df_1 = spark.sql("""
    SELECT s.*, i.type_indexed, i.brand_indexed, i.size_value, i.size_unit_encoded
    FROM sales s
    LEFT JOIN item i ON s.code = i.code
""")
combined_df_1.createOrReplaceTempView("combined_1")



# --- 2. Join Supermarket and Promotion Data (creates final featured DF) ---
combined_featured_df = spark.sql("""
    SELECT 
        c1.*, 
        sm.postal_code_indexed,
        P.feature_indexed AS promo_feature_indexed,
        p.display_indexed as promo_display_indexed
    FROM combined_1 c1
    
    -- LEFT JOIN to Supermarkets
    LEFT JOIN supermarkets sm 
        ON c1.supermarket = sm.supermarket
        
    -- LEFT JOIN to Promotion Data to get promo_feature_indexed
    LEFT JOIN promotion P 
        ON c1.code = P.code AND
           c1.supermarket = P.supermarkets
          
""")
combined_featured_df.createOrReplaceTempView("combined_featured_df_cleaned") # Renaming the view for the final step

print("✅ Step 2/3: Added Supermarket and Promotion features.")

In [0]:
from pyspark.sql.functions import lit, count, when, col
from pyspark.sql.types import DoubleType

# --- 1. INSPECT NULLS (Recommended Check) ---
# Check for nulls in the new item features introduced by the join
print("--- Null Counts Before Imputation ---")
combined_featured_df.select(
    [count(when(col(c).isNull(), c)).alias(c) for c in 
     ["type_indexed", "brand_indexed", "size_value", "promo_feature_indexed", "promo_display_indexed"]]
).show()


# --- 2. NULL HANDLING (Imputation) ---

# Define fill values:
# - Indexed Features (Categorical): Fill with a unique, non-existent index (e.g., 9999.0) 
#   to represent 'UNKNOWN/MISSING' category. These must be double/float type.
FILL_INDEXED = 9999.0 
# - size_value (Continuous): Fill with 0.0. 
FILL_SIZE_VALUE = 0.0

# Apply the null filling to the item-related features
combined_featured_df_cleaned = combined_featured_df.fillna({
    # Fill indexed features with the sentinel value 
    'type_indexed': FILL_INDEXED,
    'brand_indexed': FILL_INDEXED,
    # Fill continuous feature with 0.0
    'size_value': FILL_SIZE_VALUE,
    #'postal_code_indexed': FILL_INDEXED,
    'promo_feature_indexed': FILL_INDEXED,
    'promo_display_indexed': FILL_INDEXED
})


# --- 3. VERIFY AND PROCEED ---

# Ensure the columns are cast back to the expected type if necessary (Spark often handles this)
combined_featured_df_cleaned = combined_featured_df_cleaned\
    .withColumn("type_indexed", col("type_indexed").cast(DoubleType())) \
    .withColumn("brand_indexed", col("brand_indexed").cast(DoubleType()))

print("--- Verification: Null Counts After Imputation ---")
combined_featured_df_cleaned.select(
    [count(when(col(c).isNull(), c)).alias(c) for c in ["type_indexed", "brand_indexed", "size_value"]]
).show()

# You can now proceed with your scenario analysis using the 'combined_featured_df_cleaned' DataFrame.

In [0]:
display(combined_featured_df_cleaned)

In [0]:
from pyspark.ml.regression import RandomForestRegressionModel 

# Use the Databricks mount point for the model
model_path = "/mnt/model/sales_rf_model_v1"

try:
    loaded_model = RandomForestRegressionModel.load(model_path)

except Exception as e:
    print(f"Could not load model. Error: {e}")

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col, lit

# --- ASSUMPTIONS & PREREQUISITES ---
# TARGET_PRODUCT_CODE: An item 'code' (e.g., a high seller).
# -----------------------------------

# 1. DEFINE YOUR FEATURE LIST (CRITICAL STEP)
# This list MUST match the input columns used for model training EXACTLY.
FEATURE_COLUMNS = [
    # Sales/Temporal Features
    "hour_sin", "hour_cos", 
    "voucher",
    "province_1", "province_2", 
    "cycle_day_1", "cycle_day_2", "cycle_day_3", "cycle_day_4",
    "cycle_day_5", "cycle_day_6", "cycle_day_7",
    
    # Item Features (Where the NaN is likely coming from due to failed extraction or join)
    "size_value",
    "size_unit_encoded", 
    "brand_indexed",     
    "type_indexed",      
    
    # Supermarket/Promotion Features
    "postal_code_indexed", 
    "promo_feature_indexed",
    "promo_display_indexed"
]



# --- SCENARIO SETUP ---

TARGET_PRODUCT_CODE = 7546203734 # Use a product code you know is relevant
baseline_df = combined_featured_df_cleaned.filter(col("code") == TARGET_PRODUCT_CODE).limit(100)

# 2. Create the BASELINE Scenario (All promotions OFF)
baseline_scenario = baseline_df.withColumn("voucher", lit(0)) \
                               .withColumn("promo_feature_indexed", lit(0)) \
                               .withColumn("Scenario", lit("Baseline"))

# 3. Create the VOUCHER Scenario (Voucher ON)
voucher_scenario = baseline_df.withColumn("voucher", lit(1)) \
                              .withColumn("promo_feature_indexed", lit(0)) \
                              .withColumn("Scenario", lit("Voucher"))

# 4. Combine Scenarios
combined_scenarios = baseline_scenario.unionAll(voucher_scenario)


# --- FEATURE ASSEMBLING & PREDICTION ---

# 5. Assemble Features: This creates the single 'features' vector column required by the model.
assembler = VectorAssembler(
    inputCols=FEATURE_COLUMNS,
    outputCol="features"
)
combined_scenarios_vect = assembler.transform(combined_scenarios)

# 6. Run Prediction using the loaded model
promotion_analysis_df = loaded_model.transform(combined_scenarios_vect)

# 7. Aggregate Results
promotion_results = promotion_analysis_df.groupBy("Scenario") \
    .agg({"prediction": "avg"}) \
    .withColumnRenamed("avg(prediction)", "Average_Predicted_Sales") \
    .orderBy("Average_Predicted_Sales", ascending=False)

print("\n--- Promotion Effectiveness 'What-If' Analysis Results ---")
promotion_results.show()

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col, lit

# --- ASSUMPTIONS & PREREQUISITES ---
# Assuming 'loaded_model' (your best model) and 'combined_featured_df_cleaned' 
# (your final feature DataFrame) are available.
# -----------------------------------

# 1. Reuse the base filtered data for a specific product
TARGET_PRODUCT_CODE = 7546203734
# We use a base DataFrame to ensure all non-promotion features (time, location, size) are realistic
base_df = combined_featured_df_cleaned.filter(col("code") == TARGET_PRODUCT_CODE).limit(100)

# 2. Define All Scenarios (assuming 0.0 is 'Not on Promo' and 1.0 is a typical promotion index)

# A. 01. Baseline Scenario (All promotions OFF)
baseline_scenario = base_df.withColumn("voucher", lit(0)) \
                          .withColumn("promo_feature_indexed", lit(0.0)) \
                          .withColumn("promo_display_indexed", lit(0.0)) \
                          .withColumn("Scenario", lit("01_Baseline_OFF"))

# B. 02. Voucher Scenario (Voucher ON, others OFF)
voucher_scenario = base_df.withColumn("voucher", lit(1)) \
                         .withColumn("promo_feature_indexed", lit(0.0)) \
                         .withColumn("promo_display_indexed", lit(0.0)) \
                         .withColumn("Scenario", lit("02_Voucher_Only"))

# C. 03. Feature Promo Scenario (e.g., Item featured in a flyer, others OFF)
feature_scenario = base_df.withColumn("voucher", lit(0)) \
                         .withColumn("promo_feature_indexed", lit(1.0)) \
                         .withColumn("promo_display_indexed", lit(0.0)) \
                         .withColumn("Scenario", lit("03_Feature_Only"))

# D. 04. Display Promo Scenario (e.g., End Cap Display ON, others OFF)
display_scenario = base_df.withColumn("voucher", lit(0)) \
                         .withColumn("promo_feature_indexed", lit(0.0)) \
                         .withColumn("promo_display_indexed", lit(1.0)) \
                         .withColumn("Scenario", lit("04_Display_Only"))

# E. 05. Maximum Scenario (All promotions ON)
max_scenario = base_df.withColumn("voucher", lit(1)) \
                      .withColumn("promo_feature_indexed", lit(1.0)) \
                      .withColumn("promo_display_indexed", lit(1.0)) \
                      .withColumn("Scenario", lit("05_Max_Combined"))

# 3. Combine All Scenarios
combined_scenarios = baseline_scenario.unionAll(voucher_scenario) \
                                      .unionAll(feature_scenario) \
                                      .unionAll(display_scenario) \
                                      .unionAll(max_scenario)

# 4. Feature Assembler (MUST match original training features)
FEATURE_COLUMNS = [
    "hour_sin", "hour_cos", "voucher", "province_1", "province_2", 
    "cycle_day_1", "cycle_day_2", "cycle_day_3", "cycle_day_4",
    "cycle_day_5", "cycle_day_6", "cycle_day_7",
    "size_value", "size_unit_encoded", "brand_indexed", "type_indexed",      
    "postal_code_indexed", "promo_feature_indexed", "promo_display_indexed"
]

assembler = VectorAssembler(
    inputCols=FEATURE_COLUMNS,
    outputCol="features"
)
combined_scenarios_vect = assembler.transform(combined_scenarios)

# 5. Run Prediction using the loaded model (assuming it's loaded as loaded_model)
promotion_analysis_df = loaded_model.transform(combined_scenarios_vect)

# 6. Aggregate Results
promotion_results = promotion_analysis_df.groupBy("Scenario") \
    .agg({"prediction": "avg"}) \
    .withColumnRenamed("avg(prediction)", "Average_Predicted_Sales") \
    .orderBy("Scenario", ascending=True) # Order by Scenario number for clean viewing

print("\n--- Expanded Promotion Effectiveness Analysis Results ---")
promotion_results.show()